## Pipeline Orchestrations Simple Tutorial

This tutorial provides a quick set of methods and examples regarding Wallaroo Connections and Wallaroo ML Workload Orchestration.  For full details, see the Wallaroo Documentation site.

Wallaroo provides data connections, orchestrations, and tasks to provide organizations with a method of creating and managing automated tasks that can either be run on demand, on a regular schedule, or as a service so they respond to requests.

| Object | Description |
|---|---|
| Orchestration | A set of instructions written as a python script with a requirements library.  Orchestrations are uploaded to the Wallaroo instance |
| Task | An implementation of an orchestration.  Tasks are run either once when requested, on a repeating schedule, or as a service. |
| Connection | Definitions set by MLOps engineers that are used by other Wallaroo users for connection information to a data source.  Usually paired with orchestrations. |

## Tutorial Goals

The tutorial will demonstrate the following:

1. Create a Wallaroo connection to retrieving information from an external source.
1. Create a Wallaroo connection to store information into an external source.
1. Upload Wallaroo ML Workload Orchestration.
1. Run the orchestration once as a Run Once Task and verify that the information was saved to the external data store.
1. Schedule the orchestration as a Scheduled Task and verify that the information was saved to the external data store.

## Tutorial Required Libraries

The following libraries are required for this tutorial, and included by default in a Wallaroo instance's JupyterHub service.

* **IMPORTANT NOTE**:  These libraries are already installed in the Wallaroo JupyterHub service.

* [wallaroo](https://pypi.org/project/wallaroo/):  The Wallaroo SDK.
* [pandas](https://pypi.org/project/pandas/): The pandas data analysis library.
* [pyarrow](https://pypi.org/project/pyarrow/): The Apache Arrow Python library.

These are specified in the `requirements.txt` file for this tutorial.

## Connect to the Wallaroo Instance

The first step is to connect to a Wallaroo instance.  We'll load the libraries and set our client connection settings

### Workspace, Model and Pipeline Setup

For this tutorial, we'll create a workspace, upload our sample model and deploy a pipeline.  We'll perform some quick sample inferences to verify that everything it working.

In [ ]:
import wallaroo
from wallaroo.object import EntityNotFoundError

# to display dataframe tables
from IPython.display import display
# used to display dataframe information without truncating
import pandas as pd
pd.set_option('display.max_colwidth', None)
import pyarrow as pa

import os
# Used for the Wallaroo SDK version 2023.1
os.environ["ARROW_ENABLED"]="True"

import requests
import time
import datetime
import calendar

In [ ]:
# Login through local Wallaroo instance

# wl = wallaroo.Client()

# # SSO login through keycloak

# wallarooPrefix = "YOUR PREFIX"
# wallarooSuffix = "YOUR PREFIX"


# wallarooPrefix = "doc-test"
# wallarooSuffix = "wallaroocommunity.ninja"


# wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
#                     auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
#                     auth_type="sso")

# os.environ["WALLAROO_SDK_CREDENTIALS"] = './creds.json.example'

wallarooPrefix="doc-test"
wallarooSuffix="wallaroocommunity.ninja"

# # wallarooPrefix="product-uat-ee"
# # wallarooSuffix="wallaroocommunity.ninja"

# wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
#                     auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
#                     auth_type="sso")

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
                    auth_type="sso")

In [ ]:
# Setting variables for later steps

workspace_name = 'simpleorchestrationworkspace'
pipeline_name = 'simpleorchestrationpipeline'
model_name = 'simpleorchestrationmodel'
model_file_name = './models/rf_model.onnx'
inference_connection_name = "external_inference_connection"
inference_results_connection_name = "inference_results_connection"

### Helper Methods

The following helper methods are used to either create or get workspaces and pipelines.

In [ ]:
# helper methods to retrieve workspaces and pipelines

def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

### Create the Workspace and Pipeline

We'll now create our workspace and pipeline for the tutorial.  If this tutorial has been run previously, then this will retrieve the existing ones with the assumption they're for us with this tutorial.

We'll set the retrieved workspace as the current workspace in the SDK, so all commands will default to that workspace.

In [ ]:
workspace = get_workspace(workspace_name)
wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)

### Upload the Model and Deploy Pipeline

We'll upload our model into our sample workspace, then add it as a pipeline step before deploying the pipeline to it's ready to accept inference requests.

In [ ]:
# Upload the model

housing_model_control = wl.upload_model(model_name, model_file_name).configure()

# Add the model as a pipeline step

pipeline.add_model_step(housing_model_control)

In [ ]:
#deploy the pipeline
pipeline.deploy()

## Create Connections

We will create both the data source and the results location connections via the Wallaroo client command `create_connection`.

Connections are created with the Wallaroo client command [`create_connection`](https://staging.docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-ml-workload-orchestration/#create-orchestration) with the following parameters.

| Parameter | Type | Description |
| --- | --- | ---|
| **name** | string (Required) | The name of the connection. This must be unique - if submitting the name of an existing connection it will return an error. |
| **type** | string (Required) | The user defined type of connection. |
| **details** | Dict (Requires) | User defined configuration details for the data connection.  These can be `{'username':'dataperson', 'password':'datapassword', 'port': 3339}`, or `{'token':'abcde123==', 'host':'example.com', 'port:1234'}`, or other user defined combinations.  |

The data source connection will be named 

We'll create the connection named `houseprice_arrow_table`, set it to the type `HTTPFILE`, and provide the details as `'host':'https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/20230314_2023.2_updates/wallaroo-testing-tutorials/houseprice-saga/data/xtest-1k.arrow?raw=true'` - the location for our sample Arrow table inference input.

### Create Connection

Connections are created with the Wallaroo client command [`create_connection`](https://staging.docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-dataconnectors/#create-data-connection) with the following parameters.

| Parameter | Type | Description |
| --- | --- | ---|
| **name** | string (Required) | The name of the connection. This must be unique - if submitting the name of an existing connection it will return an error. |
| **type** | string (Required) | The user defined type of connection. |
| **details** | Dict (Requires) | User defined configuration details for the data connection.  These can be `{'username':'dataperson', 'password':'datapassword', 'port': 3339}`, or `{'token':'abcde123==', 'host':'example.com', 'port:1234'}`, or other user defined combinations.  |

inference_connection_name = "external_inference_connection"
inference_results_connection_name = "inference_results_connection"

We'll create the connection named `external_inference_connection` to retrieve the data used for the inference, then the inference results storage location will be set to a local file via the name `inference_results_connection`.  The local file will be set to the current epoch time as the prefix.

In [ ]:
wl.create_connection(inference_connection_name, 
                  "HTTPFILE", 
                  {'host':'https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/20230314_2023.2_updates/wallaroo-testing-tutorials/houseprice-saga/data/xtest-1k.arrow?raw=true'}
                  )

In [ ]:
import calendar

# datetime.datetime.now()

seconds = calendar.timegm(datetime.datetime.now().timetuple())

wl.create_connection(inference_results_connection_name, 
                  "LOCALFILE", 
                  {'location':f'./{seconds}_inference_result.df'}
                  )

### List Data Connections

The Wallaroo Client [`list_connections()`](https://staging.docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-dataconnectors/#list-data-connections) method lists all connections for the Wallaroo instance.

We'll retrieve the list of connections and display them.


In [ ]:
wl.list_connections()

### Get Connection by Name

The Wallaroo client method `get_connection(name)` retrieves the connection that matches the `name` parameter.  We'll retrieve our two connections and store them as `inference_source_connection` and `inference_results_connection`.

In [ ]:
inference_source_connection = wl.get_connection(name="external_inference_connection")
display(inference_source_connection)

In [ ]:
inference_results_connection = wl.get_connection(name="inference_results_connection")
display(inference_results_connection)

### Add Connection to Workspace

The method Workspace [`add_connection(connection_name)`](https://staging.docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-dataconnectors/#add-data-connection-to-workspace) adds a Data Connection to a workspace, and takes the following parameters.

| Parameter | Type | Description |
| --- | --- | ---|
| **name** | string (Required) | The name of the Data Connection |

We'll add both connections to our sample workspace, then list the connections available to the workspace to confirm.

In [ ]:
workspace.add_connection("inference_results_connection")
workspace.add_connection("external_inference_connection")
workspace.list_connections()

## Wallaroo ML Workload Orchestration Example

With the pipeline deployed and our connections set, we will now generate our ML Workload Orchestration.  See the [Wallaroo ML Workload Orchestrations guide](https://staging.docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-ml-workload-orchestration/) for full details.

Orchestrations are uploaded to the Wallaroo instance as a ZIP file with the following requirements:

{{<table "table table-striped table-bordered">}}
| Parameter | Type | Description |
|---|---|---|
| **User Code** | (*Required*) Python script as `.py` files | If `main.py` exists, then that will be used as the task entrypoint. Otherwise, the **first** `main.py` found in any subdirectory will be used as the entrypoint. |
| Python Library Requirements | (*Optional*) `requirements.txt` file in the [requirements file format](https://pip.pypa.io/en/stable/reference/requirements-file-format/).  A standard Python requirements.txt for any dependencies to be provided in the task environment. The Wallaroo SDK will already be present and **should not be included in the requirements.txt**. Multiple requirements.txt files are not allowed. |
| Other artifacts | &nbsp; | Other artifacts such as files, data, or code to support the orchestration.
{{</table>}}

For our example, our orchestration will:

1. Use the `inference_results_connection` to open a HTTP Get connection to the inference data file and use it in an inference request in the deployed pipeline.
1. Submit the inference results to the location specified in the `external_inference_connection`.

This sample script is stored in `remote_inference/main.py`, and packaged into the orchestration as `./remote_inference/remote_inference.zip`.  We'll display the steps in uploading the orchestration to the Wallaroo instance.

Note that the orchestration assumes that the pipeline is already deployed.

### Upload the Orchestration

Orchestrations are uploaded with the Wallaroo client `upload_orchestration(path)` method with the following parameters.

| Parameter | Type | Description |
| --- | --- | ---|
| **path** | string (Required) | The path to the ZIP file to be uploaded. |

Once uploaded, the deployment will be prepared and any requirements will be downloaded and installed.


For this example, the orchestration `./remote_inference/remote_inference.zip` will be uploaded and saved to the variable `orchestration`.

In [ ]:
orchestration = wl.upload_orchestration(path="./remote_inference/remote_inference.zip")

### Orchestration Status

We will loop until the uploaded orchestration's `status` displays `ready`.

In [27]:
while orchestration.status() != 'ready':
    print(orchestration.status())
    time.sleep(5)

[]

### Run Orchestration Once

Orchestrations are run either once or run on a schedule.  We'll do both 

In [ ]:
wl.list_orchestrations()

## Task Management Tutorial

Once an Orchestration has the status `ready`, it can be run as a task.  Tasks have three run options.

| Type | SDK Call |  How triggered |
|---|---|:---|
| Once       | `orchestration.run_once(json_args)` | User makes one api call. Task runs once and exits.| Single batch, experimentation. |
| Scheduled  | `orchestration.run_scheduled(name, schedule, timeout, json_args)` | User provides schedule. Task runs exits whenever schedule dictates. | Recurrent batch. |

### Task Run Once

Tasks are generated and run once with the Orchestration `run_once(arguments)` method.  Any arguments for the orchestration are passed in as a `Dict`.  If there are no arguments, then an empty set `{}` is passed.

In [ ]:
# Example: run once

import datetime
task_start = datetime.datetime.now()
orchestration.run_once()
task = orchestration.run_once({})
task

### Task Status

The list of tasks in the Wallaroo instance is retrieves through the Wallaroo Client `list_tasks()` method.  This returns an array list of the following.

| Parameter | Type | Description |
| --- | --- | ---|
| **id** | string | The UUID identifier for the task. |
| **status** | enum | The status of the task.  Values are: <br><ul><li>`pending`: The task has not been started.</li><li>`started`: The task has been scheduled to execute.</li><li>`pending_kill`: The task kill command has been issued and the task is scheduled to be stopped.</li></ul> |
| **type** | string | The type of the task.  Values are: <br><ul><li>`Temporary Run`: The task runs once then stop.</li><li>`Scheduled Run`: The task repeats on a `cron` like schedule.</li><li>`Service Run`: The task runs as a service and executes when its service port is activated. |
| **created at** | DateTime | The date and time the task was started. |
| **updated at** | DateTime | The date and time the task was updated. |

For this example, the status of the previously created task will be generated, then looped until it has reached status `started`.

In [ ]:
while task.status() != "started":
    display(task.status())
    time.sleep(5)

In [ ]:
wl.list_tasks()

In [ ]:
task.status()

In [ ]:
task_scheduled = orchestration.run_scheduled(name="schedul_test", timeout=600, schedule="*, 5, *, *, *", json_args={})

In [ ]:
task_scheduled.status()

In [ ]:
wl.list_tasks()

In [ ]:
orchestration.run_scheduled()

### Task Results

We can view the inferences from our logs and verify that new entries were added from our task.  In our case, we'll assume the task once started takes about 1 minute to run (deploy the pipeline, run the inference, undeploy the pipeline).  We'll add in a wait of 1 minute, then display the logs during the time period the task was running.

In [ ]:
# time.sleep(60)

task_end = datetime.datetime.now()
display(task_end)

pipeline.logs(start_datetime = task_start, end_datetime = task_end)

### Kill a Task

Killing a task removes the schedule or removes it from a service.  Tasks are killed with the Task `kill()` method, and returns a message with the status of the kill procedure.

If necessary, all tasks can be killed through the following script.

* **IMPORTANT NOTE**:  This command will kill all running tasks - scheduled or otherwise.  Only use this if required.

```python
# Kill all tasks
for t in wl.list_tasks(): t.kill()
```

In [ ]:
task.kill()

In [ ]:
task.status()